# Příprava datové sady pro dolovací algoritmy
*Připravte 2 varianty datové sady vhodné pro dolovací algoritmy.*

Datová sada: Tučňáci \
Dolovací úloha: Klasifikace druhů tučňáků na základě ostatních atributů.

In [90]:
# potřebné moduly
import pandas as pd
import numpy as np
import copy

# načtení dat a uložení do dataframu
csv_file="datasets/dataset-penguins/penguins_lter.csv"
df = pd.read_csv(csv_file)

## Odstranění irelevantních atributů
*Odstraňte z datové sady atributy, které jsou pro danou dolovací úlohu irelevantní.*

In [91]:
# odstranění 9 z původních 17 atributů
df_tmp = df.drop(['studyName', 'Sample Number', 'Region', 'Island', 'Stage', 'Clutch Completion', 'Date Egg', 'Individual ID', 'Comments'], axis=1)
#print(df_tmp.columns)

## Úprava chybějících hodnot
*Vypořádejte se s chybějícími hodnotami. Pro odstranění těchto hodnot využijte alespoň dvě různé metody pro odstranění chybějících hodnot.*

In [92]:
# atribut s pohlavím obsahuje chybnou hodnotu ('.') – nejprve ji odstraníme
df_tmp.loc[(df_tmp['Sex'] != "MALE") & (df_tmp['Sex'] != "FEMALE"), 'Sex'] = np.NaN

# identifikace atributů s chybějícími hodnotami
missing_values = {k: v for k, v in df_tmp.isna().sum().items() if v > 0}

# odstranění záznamů, kterým chybějí hodnoty u všech identifikovaných atributů
# konkrétně se jedná o všechny numerické atributy a kategorický atribut pohlaví
df_tmp = df_tmp.drop(df_tmp[df_tmp[missing_values.keys()].isna().all(axis=1)].index)

# # podmnožiny dataframů podle druhu tučňáka
# peng1 = df_tmp.loc[df_tmp['Species'] == 'Adelie Penguin (Pygoscelis adeliae)'].copy()
# peng2 = df_tmp.loc[df_tmp['Species'] == 'Chinstrap penguin (Pygoscelis antarctica)'].copy()
# peng3 = df_tmp.loc[df_tmp['Species'] == 'Gentoo penguin (Pygoscelis papua)'].copy()

# medians or means pro druhy a pohlavi tucnaka
def select_sex(m_values: dict[tuple[str,str], float], species: str, weight = int | float):
    """Vraci odhadovane pohlavi tucnaka na zaklade druhu a vahy. Je potreba parametrem predat predpocitany slovnik medianu nebo strednich hodnot tucnaku."""
    male_weight = m_values[(species, "MALE")]
    female_weight = m_values[(species, "FEMALE")]

    male_distance = abs(weight - male_weight)
    female_distance = abs(weight - female_weight)

    return "MALE" if male_distance < female_distance else "FEMALE"

# def get_distance(m_values: dict[tuple[str,str], float], species: str, sex:str, weight = int | float):
#     return abs(m_values[(species, sex)] - weight)

# doplnění chybějících hodnot u numerických atributů (pozn. modus nemá smysl uvažovat, viz nejčastější hodnoty atributů zjištěné explorační analýzou)
# 1. varianta: nahrazení mediánem
df_var1 = df_tmp.copy()
for col in missing_values.keys():
    if col != "Sex":
        df_var1[col] = df_var1[col].replace(np.nan, df_var1[col].median())



# seznam medianu vahy podle druhu a podle pohlavi
# prvni hodnota v tuple je druh a druha je pohlavi
medians : dict[tuple[str, str], float] = {}

for species in df_var1["Species"].unique():
    for sex in ["MALE", "FEMALE"]:
        filtered_df = df_var1[(df_var1['Sex'] == sex) & (df_var1["Species"] == species)]
        mass_median = filtered_df.loc[:, ["Body Mass (g)"]].median()
        
        medians[(species,sex)] = mass_median
        

# for index, row in df_var1.iterrows():
#     df_var1.loc[index, 'Sex'] = select_sex(medians, row['Species'], row['Body Mass (g)'])
    # tady to pada nevim proc :'( ale nahradilo by to pohlavi odhadovanym pohlavim, jeste by se tu muselo pohlidat aby k nahrazeni doslo jen u chybejicich


# 2. varianta: nahrazení průměrem
df_var2 = df_tmp.copy()
for col in missing_values.keys():
    if col != "Sex":
        df_var2[col] = df_var2[col].replace(np.nan, df_var2[col].mean())

# tady by byl podobny kod jako nahore jen by se pouzil mean, ale na to uz neni cas bohuzel


## Úprava odlehlých hodnot
*Vypořádejte se s odlehlými hodnotami, jsou-li v datové sadě přítomny.*

In [93]:
# žádné odlehlé hodnoty nebyly při explorační analýze detekovány

## Diskretizace numerických atributů
*Pro jednu variantu datové sady proveďte diskretizaci numerických atributů tak, aby výsledná datová sada byla vhodná pro algoritmy, které vyžadují na vstupu kategorické atributy.*

In [94]:
# výběr numerických (kvantitativních) atributů
numeric_columns = df_var1.select_dtypes(include=['int64', 'float64']).columns

# diskretizace hodnot přes ekvifrekvenční intervaly (intervaly mají přibližně stejnou četnost)
for col in numeric_columns:
    df_var1[col] = pd.qcut(df_var1[col], q=4)

print("Procentuální rozložení hodnot po diskretizaci: ")
for c in numeric_columns:
    print(f"Sloupec: {c}\n{df_var1[c].value_counts(normalize=True)}\n")

Procentuální rozložení hodnot po diskretizaci: 
Sloupec: Culmen Length (mm)
(44.45, 48.5]                   0.254386
(32.099000000000004, 39.225]    0.251462
(39.225, 44.45]                 0.248538
(48.5, 59.6]                    0.245614
Name: Culmen Length (mm), dtype: float64

Sloupec: Culmen Depth (mm)
(13.099, 15.6]    0.257310
(15.6, 17.3]      0.254386
(17.3, 18.7]      0.251462
(18.7, 21.5]      0.236842
Name: Culmen Depth (mm), dtype: float64

Sloupec: Flipper Length (mm)
(171.999, 190.0]    0.289474
(213.0, 231.0]      0.248538
(197.0, 213.0]      0.236842
(190.0, 197.0]      0.225146
Name: Flipper Length (mm), dtype: float64

Sloupec: Body Mass (g)
(2699.999, 3550.0]    0.260234
(3550.0, 4050.0]      0.254386
(4750.0, 6300.0]      0.248538
(4050.0, 4750.0]      0.236842
Name: Body Mass (g), dtype: float64

Sloupec: Delta 15 N (o/oo)
(8.306, 8.652]                0.266082
(7.630999999999999, 8.306]    0.251462
(9.141, 10.025]               0.251462
(8.652, 9.141]            

## Transformace kategorických atributů
*Pro druhou variantu datové sady proveďte vhodnou transformaci kategorických atributů na numerické atributy. Dále pak proveďte normalizaci numerických atributů, které má smysl normalizovat. Výsledná datová sada by měla být vhodná pro metody vyžadující numerické vstupy.*

In [95]:
# normalizace numerických atributů (min-max)
numeric_columns = df_var2.select_dtypes(include=['int64', 'float64']).columns

for col in numeric_columns:
    df_var2[col] = (df_var2[col] - df_var2[col].min())/(df_var2[col].max() - df_var2[col].min())

# nárůst dimenzí přidáním sloupců pro kategoriální hodnoty atributů pohlaví a druhu tučňáků 
for val in df_var2['Sex'].unique():
    df_var2[val] = [1 if i == val else 0 for i in df_var2['Sex']]
df_var2.drop('Sex', inplace=True, axis=1)

for val in df_var2['Species'].unique():
    df_var2[val] = [1 if i == val else 0 for i in df_var2['Species']]
df_var2.drop('Species', inplace=True, axis=1)

## Export upravených datových sad
*Export obou výsledných variant upravené datové sady do formátu csv. Pro zmenšení velikosti odevzdávaného archivu je možné odevzdat pouze prvních 50 řádků těchto souborů.*

In [96]:
df_var1.to_csv("penguins_classif_var1.csv")
df_var2.to_csv("penguins_classif_var2.csv")